# Load Packages

In [1]:
import numpy as np

#!pip install memory_profiler
import memory_profiler
%load_ext memory_profiler

# Load Data

In [2]:
with np.load('data/demo1.npz') as data:
    x = data['px'][:, 0]

# Find truncation order m
The task is to find the first $m^*$ whereas 

$$
|w_{m^*}| \geq \tau \; \wedge \; |w_{m^*+1}| < \tau
$$

(see "Fixed-Width Window Fracdiff" in Lopez, 2018, pp. 80--84)

In [3]:
# numpy version
def find_truncation_order1(d, tau=1e-5, mmax=20000):
    w = np.empty((mmax+1,))
    w[0] = 1
    for k in range(1, mmax+1):
        w[k] = -w[k-1] * ((d - k + 1) / k)
        if np.abs(w[k]) < tau:
            break
    return k-1, w[:k]

In [4]:
# list version
def find_truncation_order2(d, tau=1e-5, mmax=20000):
    w = [1]
    for k in range(1, mmax+1):
        wk = -w[-1] * ((d - k + 1) / k)
        if abs(wk) < tau:
            break
        w.append(wk)
    return k-1, w

# Speed
The `list` version is much faster

In [5]:
d = 0.345
%timeit mstar, w = find_truncation_order1(d)
%timeit mstar, w = find_truncation_order2(d)

8.01 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
936 µs ± 129 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Memory

In [6]:
d = 0.345
%memit mstar, w = find_truncation_order1(d)
%memit mstar, w = find_truncation_order2(d)

peak memory: 62.33 MiB, increment: -0.14 MiB
peak memory: 62.11 MiB, increment: 0.01 MiB


# References
* Prado, M.L. de, 2018. Advances in Financial Machine Learning, 1st ed. Wiley.